### Script para bajar datos de F1

In [15]:
import fastf1
import pandas as pd
from typing import Literal


In [17]:

def get_f1_session_data(year: int, session_type: Literal['S', 'R', 'Q'], output_file: str = None) -> pd.DataFrame:
    """
    Traer información de eventos de F1 para un determinado año y tipo de sesión (carrera, qualy o sprint) year and session type y exportarlo a un archivo CSV.
    
    Args:
        year (int): el año de interés (ej., 2024).
        session_type (str): tipo de sesión que puede ser 'S' (Sprint), 'R' (Race), 'Q' (Qualifying).
        output_file (str, optional): ruta para guardar el archivo resultante. Defaults to 'f1_{year}_{session_type}.csv'.
    
    Returns:
        pd.DataFrame: el archivo obtenido.
    
    Raises:
        ValueError: Falla si session_type no es alguna de las opciones establecidas ('S', 'R', o 'Q').
    """
    # Validate session_type
    accepted_sessions = ['S', 'R', 'Q']
    if session_type not in accepted_sessions:
        raise ValueError(f"Esperaba alguno de estos valores: {accepted_sessions} y se obtuvo {session_type}")

    
    # Get the event schedule for the year
    schedule = fastf1.get_event_schedule(year)
    race_events = schedule[schedule['EventFormat'] != 'testing']  # Exclude testing events
    
    # Initialize an empty DataFrame
    all_data = pd.DataFrame()
    
    # Loop through all events
    for index, event in race_events.iterrows():
        event_name = event['EventName']
        print(f"Processing {event_name} - {session_type}...")
        
        try:
            # Load the session with telemetry and weather
            session = fastf1.get_session(year, event_name, session_type)
            session.load(telemetry=True, weather=True)
            
            # Get lap data
            laps = session.laps
            
            # Add event and session metadata
            laps['EventName'] = event_name
            laps['SessionType'] = session_type
            laps['EventDate'] = event['EventDate']
            
            # Add weather data (merge based on lap timestamps)
            weather = session.weather_data
            if not weather.empty:
                weather['Time'] = weather['Time'].dt.total_seconds()
                laps['LapStartTime'] = laps['LapStartTime'].dt.total_seconds()
                # Approximate match: assign weather data to closest lap time
                laps = pd.merge_asof(
                    laps.sort_values('LapStartTime'),
                    weather[['Time', 'AirTemp', 'TrackTemp', 'Humidity']].sort_values('Time'),
                    left_on='LapStartTime',
                    right_on='Time',
                    direction='nearest',
                    suffixes=('', '_weather')
                )
                laps = laps.drop(columns=['Time'])  # Drop temporary weather time column
            
            # Add summarized telemetry data (Speed, Throttle, Brake)
            telemetry_summary = []
            for lap_index, lap in laps.iterrows():
                try:
                    telemetry = lap.get_telemetry()
                    telemetry_summary.append({
                        'LapNumber': lap['LapNumber'],
                        'Driver': lap['Driver'],
                        'AvgSpeed': telemetry['Speed'].mean(),
                        'MaxThrottle': telemetry['Throttle'].max(),
                        'AvgBrake': telemetry['Brake'].mean()
                    })
                except Exception:
                    # If telemetry fails, append NaN values
                    telemetry_summary.append({
                        'LapNumber': lap['LapNumber'],
                        'Driver': lap['Driver'],
                        'AvgSpeed': float('nan'),
                        'MaxThrottle': float('nan'),
                        'AvgBrake': float('nan')
                    })
            
            telemetry_df = pd.DataFrame(telemetry_summary)
            # Merge telemetry summary back into laps
            laps = pd.merge(
                laps,
                telemetry_df,
                on=['LapNumber', 'Driver'],
                how='left'
            )
            
            # Append to the master DataFrame
            all_data = pd.concat([all_data, laps], ignore_index=True)
            
        except Exception as e:
            print(f"Skipping {event_name} {session_type}: {str(e)}")
    
    # Convert timedelta columns to seconds
    time_cols = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']
    for col in time_cols:
        if col in all_data.columns:
            all_data[col] = all_data[col].dt.total_seconds()
    
    # Set default output file name if not provided
    if output_file is None:
        output_file = f"../datasets/f1_{year}_{session_type}.csv"
    
    # Export to CSV
    all_data.to_csv(output_file, index=False)
    print(f"Datos exportados a {output_file} (Filas: {all_data.shape[0]}, Columnas: {all_data.shape[1]})")
    
    return all_data



In [18]:
# Exportar sprints del 2024
get_f1_session_data(2024, 'S')
# Exportar qualis del 2024
get_f1_session_data(2024, 'Q')
# Exportar carreras del 2024
get_f1_session_data(2024, 'R')    


core           INFO 	Loading data for Chinese Grand Prix - Sprint [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


Processing Bahrain Grand Prix - S...
Skipping Bahrain Grand Prix S: Session type 'S' does not exist for this event
Processing Saudi Arabian Grand Prix - S...
Skipping Saudi Arabian Grand Prix S: Session type 'S' does not exist for this event
Processing Australian Grand Prix - S...
Skipping Australian Grand Prix S: Session type 'S' does not exist for this event
Processing Japanese Grand Prix - S...
Skipping Japanese Grand Prix S: Session type 'S' does not exist for this event
Processing Chinese Grand Prix - S...


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '44', '11', '16', '55', '4', '81', '63', '24', '20', '3', '77', '31', '18', '10', '22', '23', '2', '27', '14']
core           INFO 	Loading data for Miami Grand Prix - Sprint [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


Processing Miami Grand Prix - S...


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '3', '55', '81', '27', '22', '10', '2', '24', '63', '23', '77', '31', '44', '14', '20', '18', '4']
core           INFO 	Loading data for Austrian Grand Prix - Sprint [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


...

req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '81', '63', '10', '55', '14', '24', '20', '4', '77', '44', '22', '30', '23', '27', '11', '18', '43', '31']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Processing Abu Dhabi Grand Prix - R...


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '55', '16', '44', '63', '1', '10', '27', '14', '81', '23', '22', '24', '18', '61', '20', '30', '77', '43', '11']


Datos exportados a ../datasets/f1_2024_R.csv (Filas: 26606, Columnas: 40)


,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,...,EventName,SessionType,EventDate,Time_weather,AirTemp,TrackTemp,Humidity,AvgSpeed,MaxThrottle,AvgBrake
0,VER,1,97.284,1.0,1.0,NaT,NaT,NaN,41.266,23.616,...,Bahrain Grand Prix,R,2024-03-02,3614.115,18.2,23.8,49.0,NaN,NaN,NaN
1,LEC,16,98.271,1.0,1.0,NaT,NaT,NaN,41.549,23.635,...,Bahrain Grand Prix,R,2024-03-02,3614.115,18.2,23.8,49.0,NaN,NaN,NaN
2,RUS,63,99.228,1.0,1.0,NaT,NaT,NaN,42.099,23.694,...,Bahrain Grand Prix,R,2024-03-02,3614.115,18.2,23.8,49.0,NaN,NaN,NaN
3,NOR,4,102.168,1.0,1.0,NaT,NaT,NaN,43.309,24.244,...,Bahrain Grand Prix,R,2024-03-02,3614.115,18.2,23.8,49.0,NaN,NaN,NaN
4,HAM,44,103.122,1.0,1.0,NaT,NaT,NaN,43.609,24.393,...,Bahrain Grand Prix,R,2024-03-02,3614.115,18.2,23.8,49.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26601,GAS,10,90.865,58.0,2.0,NaT,NaT,17.995,38.878,33.992,...,Abu Dhabi Grand Prix,R,2024-12-08,8577.487,25.9,29.3,60.0,NaN,NaN,NaN
26602,HUL,27,90.040,58.0,2.0,NaT,NaT,18.096,38.630,33.314,...,Abu Dhabi Grand Prix,R,2024-12-08,8577.487,25.9,29.3,60.0,NaN,NaN,NaN
26603,MAG,20,85.637,57.0,5.0,NaT,NaT,17.257,37.109,31.271,...,Abu Dhabi Grand Prix,R,2024-12-08,8577.487,25.9,29.3,60.0,NaN,NaN,NaN
26604,ALO,14,88.621,58.0,3.0,NaT,NaT,17.619,38.418,32.584,...,Abu Dhabi Grand Prix,R,2024-12-08,8577.487,25.9,29.3,60.0,NaN,NaN,NaN
